In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('../models/ocsvm_df_test_cleaned_pred.csv',sep=';', index_col=0)

In [12]:
df.head()

,score_samples,ind_var30,var36,var15,...,num_var29_0,num_aport_var33_ult1,TARGET,ID
50595,1278.935705,1,1,45,...,0,0,0,101019
5252,7168.393634,1,3,60,...,0,0,0,10518
21956,1069.038083,1,3,27,...,0,0,0,43966
42714,-273.994650,1,1,38,...,0,0,0,85554
18494,1281.258936,1,3,23,...,0,0,0,37070


In [4]:
df['score_samples'] = df['score_samples'] *-1

A score close to 1 indicates anomalies.  
Score much smaller than 0.5 indicates normal observations.  
If all scores are close to 0.5 then the entire sample does not seem to have clearly distinct anomalies.  

prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [5]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 5)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [6]:
from scipy.stats import ks_2samp
yhat = df['score_samples']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.4165245075859622

In [7]:
results_quintil = ks(df, target='TARGET', prob='score_samples')
results_quintil

KS is 41.5% at decile 1


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,1948.868276,45138.054344,363,2678,60.0,18.0,60.0,18.0,41.5
2,86.101036,1948.855057,90,2951,15.0,20.0,75.0,39.0,36.1
3,-752.402668,85.861656,64,2976,11.0,20.0,85.0,59.0,26.2
4,-1258.653490,-753.722914,54,2987,8.9,20.0,94.0,79.0,14.7
5,-36442.374672,-1258.655996,36,3005,5.9,21.0,100.0,100.0,0.0


In [8]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['score_samples']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['score_samples']))-1)

Roc Auc:  0.7371495056814161
Gini:  0.47429901136283226


In [9]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model9_ocsvm_fs1.csv",sep=';')